# Healthcare No Show Machine Learning

Use machine learning technique to forecast no show.

In [ ]:
import sys
sys.path.append("../..")  # add src to path to import custom modules

import os
import numpy as np
import dotenv
dotenv.load_dotenv()

import sqlalchemy
import pandas as pd
pd.set_option('display.expand_frame_repr', False)

import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset, random_split

## Load dataset

## Training a model

In [70]:
class CustomDataset(Dataset):
    def __init__(self, dataframe):
        self.dataframe = dataframe

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        features = torch.tensor(row[:-1].values, dtype=torch.float32)
        label = torch.tensor(row.iloc[-1], dtype=torch.float32)
        return features, label

In [71]:
full_dataset = CustomDataset(df)
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

In [49]:
class TransformerClassifier(nn.Module):
    """A transformer-based classifier for sequence data.
    
    This model uses a transformer encoder architecture followed by a classification layer
    to perform sequence classification tasks.
    
    Args:
        input_dim (int): Dimension of input features
        num_classes (int): Number of output classes
        d_model (int, optional): Dimension of transformer model. Defaults to 512.
        nhead (int, optional): Number of attention heads. Defaults to 8.
        num_encoder_layers (int, optional): Number of transformer encoder layers. Defaults to 3.
        dim_feedforward (int, optional): Dimension of feedforward network. Defaults to 2048.
        dropout (float, optional): Dropout rate. Defaults to 0.1.
    """
    
    def __init__(
            self, 
            input_dim: int, 
            num_classes: int, 
            d_model: int = 512, 
            nhead: int = 8, 
            num_encoder_layers: int = 3, 
            dim_feedforward: int = 2048, 
            dropout: float = 0.1
    ) -> None:
        super().__init__()
        
        # Input projection layer
        self.input_projection = nn.Linear(input_dim, d_model)
        nn.init.xavier_normal_(self.input_projection.weight)
        nn.init.constant_(self.input_projection.bias, 0.1)
        
        # Transformer encoder
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=nhead,
            dim_feedforward=dim_feedforward,
            dropout=dropout,
            batch_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(
            encoder_layer,
            num_layers=num_encoder_layers
        )
        for name, param in self.transformer_encoder.named_parameters():
            if 'weight' in name:
                nn.init.xavier_normal_(param.unsqueeze(0))
        
        # Output classifier
        self.classifier = nn.Linear(d_model, num_classes, bias=False)
        nn.init.xavier_normal_(self.classifier.weight)
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """Forward pass of the model.
        
        Args:
            x (torch.Tensor): Input tensor of shape (batch_size, seq_length, input_dim)
            
        Returns:
            torch.Tensor: Output tensor of shape (batch_size, num_classes)
        """
        # Project input to d_model dimensions
        x = self.input_projection(x)
        x = nn.ReLU()(x)
        
        # Apply transformer encoder
        x = self.transformer_encoder(x)
        
        # Classification layer
        output = self.classifier(x)
        return output

In [50]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_features = len(df.columns) - 1
n_classes = 1
model = TransformerClassifier(
    input_dim=n_features,
    num_classes=n_classes,
    num_encoder_layers=6
).to(device)
criterion = nn.BCEWithLogitsLoss()

In [51]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[30, 60, 90], gamma=0.1)
num_epochs = 100
running_loss = 0.0
for epoch in range(num_epochs):
    model.train()
    for features, labels in train_loader:
        features, labels = features.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(features.unsqueeze(1))  # Add sequence dimension
        loss = criterion(outputs.squeeze(), labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    scheduler.step()
    
    model.eval()
    with torch.no_grad():
        val_loss = 0.0
        for features, labels in val_loader:
            features, labels = features.to(device), labels.to(device)
            outputs = model(features.unsqueeze(1))
            val_loss += criterion(outputs.squeeze(), labels).item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, Val Loss: {val_loss/len(val_loader):.4f}")
    running_loss = 0.0

/tmp/ipykernel_920/1738235054.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  label = torch.tensor(row[-1], dtype=torch.float32)


Epoch [1/100], Loss: 0.5051, Val Loss: 0.4861
Epoch [2/100], Loss: 0.4974, Val Loss: 0.4990
Epoch [3/100], Loss: 0.5046, Val Loss: 0.4980
Epoch [4/100], Loss: 0.4758, Val Loss: 0.4529
Epoch [5/100], Loss: 0.4533, Val Loss: 0.4439
Epoch [6/100], Loss: 0.4493, Val Loss: 0.4451
Epoch [7/100], Loss: 0.4487, Val Loss: 0.4455
Epoch [8/100], Loss: 0.4472, Val Loss: 0.4434
Epoch [9/100], Loss: 0.4455, Val Loss: 0.4417
Epoch [10/100], Loss: 0.4441, Val Loss: 0.4417
Epoch [11/100], Loss: 0.4446, Val Loss: 0.4464
Epoch [12/100], Loss: 0.4441, Val Loss: 0.4402
Epoch [13/100], Loss: 0.4412, Val Loss: 0.4405
Epoch [14/100], Loss: 0.4409, Val Loss: 0.4445
Epoch [15/100], Loss: 0.4394, Val Loss: 0.4401
Epoch [16/100], Loss: 0.4403, Val Loss: 0.4439
Epoch [17/100], Loss: 0.4384, Val Loss: 0.4404
Epoch [18/100], Loss: 0.4360, Val Loss: 0.4400
Epoch [19/100], Loss: 0.4347, Val Loss: 0.4402
Epoch [20/100], Loss: 0.4329, Val Loss: 0.4459
Epoch [21/100], Loss: 0.4318, Val Loss: 0.4410
Epoch [22/100], Loss: 

In [72]:
accuracies = []
model.eval()
with torch.no_grad():
    for features, labels in val_loader:
        features = features.to(device)
        outputs = model(features.unsqueeze(1))
        predictions = torch.sigmoid(outputs.squeeze()).cpu().numpy()
        accuracies.append((predictions > 0.5) == labels.numpy())

print(f"Validation Accuracy: {np.concat(accuracies, axis=0).mean():.4f}")

Validation Accuracy: 0.8290


In [64]:
torch.save(model.state_dict(), "transformer_classifier.pth")